In [25]:
import os
import openai
import sys
from langchain.chat_models import ChatOpenAI
sys.path.append('../..')

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file

openai.api_key  = os.environ['OPENAI_API_KEY']

In [4]:
from langchain.document_loaders import PyPDFLoader

loaders = [
    PyPDFLoader("files/cs229/CS229_Note_Apr_8,_2020_(Linear_Regression_&_Normal_Equation).pdf"),
    PyPDFLoader("files/cs229/CS229_Note_Apr_8,_2020_(Linear_Regression_&_Normal_Equation).pdf"),
    PyPDFLoader("files/cs229/CS229_Note_Apr_13,_2020_(Locally_Weighted_Regression_&_Logistic_Regression_&_Newton's_Method).pdf"),
    PyPDFLoader("files/cs229/CS229_Note_Apr_20,_2020_(Generative_Learning_Algorithms_&_Naive_Bayes).pdf"),
    PyPDFLoader("files/cs229/CS229_Note_Apr_22,_2020_(Laplace_Smoothing_&_Kernel_Methods).pdf"),
    PyPDFLoader("files/cs229/CS229_Note_Apr_27,_2020_(Kernel_Methods_&_SVM).pdf"),
    PyPDFLoader("files/cs229/CS229_Note_Apr_29,_2020_(Neural_Networks).pdf"),
]

docs = []
for loader in loaders:
    docs.extend(loader.load())

In [5]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1500,chunk_overlap=150)

In [6]:
splits = text_splitter.split_documents(docs)

In [13]:
from langchain.vectorstores import Chroma
from langchain.embeddings.openai import OpenAIEmbeddings
persist_directory = 'docs/chroma/'
embedding = OpenAIEmbeddings()
vectordb = Chroma(persist_directory=persist_directory, embedding_function=embedding)

In [15]:
vectordb._collection.count()

0

In [19]:
vectordb.add_documents(splits)

['d47dd5a4-52c3-11ee-b36f-e073e70bf171',
 'd47dd5a5-52c3-11ee-a9cf-e073e70bf171',
 'd47dd5a6-52c3-11ee-b92c-e073e70bf171',
 'd47dd5a7-52c3-11ee-964b-e073e70bf171',
 'd47dd5a8-52c3-11ee-b8aa-e073e70bf171',
 'd47dd5a9-52c3-11ee-aebc-e073e70bf171',
 'd47dd5aa-52c3-11ee-93bd-e073e70bf171',
 'd47dd5ab-52c3-11ee-a540-e073e70bf171',
 'd47dd5ac-52c3-11ee-bd18-e073e70bf171',
 'd47dd5ad-52c3-11ee-a4b0-e073e70bf171',
 'd47dd5ae-52c3-11ee-9540-e073e70bf171',
 'd47dd5af-52c3-11ee-9399-e073e70bf171',
 'd47dd5b0-52c3-11ee-ac1e-e073e70bf171',
 'd47dd5b1-52c3-11ee-a7e6-e073e70bf171',
 'd47dd5b2-52c3-11ee-bfb2-e073e70bf171',
 'd47dd5b3-52c3-11ee-a340-e073e70bf171',
 'd47dd5b4-52c3-11ee-8cc8-e073e70bf171',
 'd47dd5b5-52c3-11ee-a18c-e073e70bf171',
 'd47dd5b6-52c3-11ee-90b3-e073e70bf171',
 'd47dd5b7-52c3-11ee-b827-e073e70bf171',
 'd47dd5b8-52c3-11ee-bae4-e073e70bf171',
 'd47dd5b9-52c3-11ee-ac07-e073e70bf171',
 'd47dd5ba-52c3-11ee-a43c-e073e70bf171',
 'd47dd5bb-52c3-11ee-81e3-e073e70bf171',
 'd47dd5bc-52c3-

In [20]:
vectordb._collection.count()

30

In [21]:
question = "What are major topics for this class?"
docs = vectordb.similarity_search(question,k=3)
len(docs)

3

In [28]:
from langchain.chains import RetrievalQA
llm = ChatOpenAI(model_name='gpt-3.5-turbo', temperature=0)

In [29]:
qa_chain = RetrievalQA.from_chain_type(
    llm,
    retriever=vectordb.as_retriever()
)

In [30]:
result = qa_chain({'query':question})

In [31]:
result

{'query': 'What are major topics for this class?',
 'result': 'The major topics for this class include:\n1. Histograms and classification\n2. Linear Support Vector Machines (LSVM)\n3. Generative Learning Algorithms\n4. Gaussian Discriminant Analysis (GDA)\n5. Logistic Regression\n6. Locally Weighted Regression\n7. Parametric and Nonparametric learning algorithms\n8. Linear Regression\n9. Batch and Stochastic Gradient Descent\n10. Normal equation\n11. Supervised Learning\n12. Classification and Regression output\n13. Objective Functions\n14. Gradient Descent algorithms'}

In [32]:
from langchain.prompts import PromptTemplate

# Build prompt
template = """Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer. Use three sentences maximum. Keep the answer as concise as possible. Always say "thanks for asking!" at the end of the answer. 
{context}
Question: {question}
Helpful Answer:"""
QA_CHAIN_PROMPT = PromptTemplate.from_template(template)


In [33]:
# Run chain
qa_chain = RetrievalQA.from_chain_type(
    llm,
    retriever=vectordb.as_retriever(),
    return_source_documents=True,
    chain_type_kwargs={"prompt": QA_CHAIN_PROMPT}
)

In [34]:
question = "Is probability a class topic?"

In [35]:
result = qa_chain({"query": question})

In [36]:
result

{'query': 'Is probability a class topic?',
 'result': "I don't know. Thanks for asking!",
 'source_documents': [Document(page_content='GenerativeLearningAlgorithmsGaussianDiscriminantAnalysisGBAGenerativeDiscriminantcomparisonNaiveBayesExampletumordetectionlogisticregressionwigDiscriminativeLearningAlgorithmlearnspcylzorlearnshowgodirectlyxyGenerativeLearningAlgorithmbeamsPCxlypays5tclasspriorPCxlyPtygenerativefeaturesclassmumPlylydiscriminativeBagesRuleWurmply111PCxlyDpcyDpoxpadpartyDplyDpcxlyoplyDGaussianDiscriminantAnalysisGDASupposeXERDdropXLconventionAssumepalyisGaussianNLFEEERdNHpixE.EEeeeaoµ', metadata={'page': 0, 'source': 'files/cs229/CS229_Note_Apr_20,_2020_(Generative_Learning_Algorithms_&_Naive_Bayes).pdf'}),
  Document(page_content="worksforallgeneralizedlinearmethodsNaiveBayesFeaturevectorxEnglishdietitconehotvealXEfoydXiIwordiappearsinemailwanttomodelplxlyply2possiblevalueofxKeyassumptionXi'sareconditionallyindependentgiverypXiXdPCX.lypCX2lxiypCXdlXiX2yJµpcx.lypixelyplxd